In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
from datetime import date
import numpy as np
import glob
import xz.open
from decimal import Decimal

In [3]:
#main role list
mainrole = pd.read_csv('../output/recommend_maintable.csv')
#mainrole

In [4]:
start_time = "2022-04-19"
end_time = date.today()

logname = 'rolelogin'

rolelogin_raw = pd.DataFrame()
for i in pd.period_range(start_time, end_time):
    data_path = glob.glob('/home/xl_daily_update/raw_data/serverlog/G1JP/' + '/*/' + logname + '.log.' + str(i) + '*', recursive = True)
    
    for j in data_path:
        with xz.open(j, 'rt', encoding = 'utf-8') as f:
            df = pd.read_csv(f, header = None, sep = '\x01', usecols = [1,9,11], names = ["datetime", "userid", "roleid"])
            rolelogin_raw = pd.concat([rolelogin_raw,df])
        

In [5]:
rolelogin_raw = rolelogin_raw[(rolelogin_raw['userid'].isin(mainrole.userid)) & rolelogin_raw['roleid'].isin(mainrole.roleid)]
# rolelogin_raw.shape

In [6]:
#rolelogin_raw = pd.read_csv("/home/chiseanchang66/R/G1/G1JP/recommand_sys/output/g1jp_rolelogin.csv")
#change the type of date from object to datetime64
rolelogin_raw['date'] = pd.to_datetime(rolelogin_raw['datetime']).dt.date
# rolelogin_raw.dtypes

In [7]:
#add column : create date of each userid, named creation_userid
rolelogin_raw['creation_userid'] = pd.to_datetime(rolelogin_raw.groupby('userid')['date'].transform('min'))
rolelogin_raw['days_from_creation'] = (pd.to_datetime(date.today()) - rolelogin_raw['creation_userid']).dt.days
#rolelogin_raw['days_from_creation'] = (pd.to_datetime(date.today()-timedelta(days = 7)) - rolelogin_raw['creation_userid']).dt.days
#rolelogin_raw.head()

In [8]:
#login days within 7,15,30 days
days_set = [7,15,30]

#extract the userid and transform to dataframe, then return the unique value of userid
login_filter = pd.DataFrame(rolelogin_raw[['userid','roleid','days_from_creation']]).drop_duplicates()
login_days_final = pd.DataFrame(rolelogin_raw[['userid','roleid']]).drop_duplicates()
for i in days_set:
    start_date = pd.to_datetime(date.today() - timedelta(days = i))
    #start_date = pd.to_datetime(date.today() - timedelta(days = i+45))
    end_date = pd.to_datetime(date.today() - timedelta(days = 1))
    #end_date = pd.to_datetime(date.today() - timedelta(days = 8))
    #date_range = pd.date_range(start_date,end_date)
    
    temp_1 = rolelogin_raw.loc[rolelogin_raw['date'].between(start_date, end_date, inclusive=True)]
    
    temp_2 = pd.DataFrame(temp_1.groupby(['userid','roleid'])['date'].nunique())
      
    login_days = pd.merge(login_filter, temp_2, how = 'outer', left_on = ['userid','roleid'], right_on = ['userid','roleid'])
    
    #temp_2.columns = ['login_days'+ '_in_' + str(i) + '_days']              
      
    login_days['days_max'] = np.where(login_days['days_from_creation'] > i, i, login_days['days_from_creation'])
    
    login_days['login_days_in_' + str(i) + '_days'] = round(login_days['date']/login_days['days_max'],2)
    
    
    login_days_final = pd.merge(login_days_final, login_days[['userid','roleid','login_days_in_' + str(i) + '_days']], how = "inner", left_on = ['userid','roleid'], right_on = ['userid','roleid'])
    
login_days_final = login_days_final.replace(np.nan , 0)
login_days_final = login_days_final[login_days_final['login_days_in_30_days'] > 0]
#print(login_days_final)
login_days_final.to_csv("../output/g1jp_user_login_days_inframe.csv", sep = ',', index = False)


In [9]:
#近30天內有登入玩家累積登入天數
user_rolelogin_record = pd.DataFrame(rolelogin_raw.groupby(['userid','roleid'])['date'].nunique()).reset_index()


user_rolelogin_record = user_rolelogin_record.rename(columns = { 'date' : 'agg_login_days'})
user_rolelogin_record =(user_rolelogin_record[user_rolelogin_record['userid'].isin(login_days_final['userid'])]).set_index('userid','roleid')
#print(user_rolelogin_record)
user_rolelogin_record.to_csv("../output/g1jp_user_agg_login_days.csv")

In [10]:
#login days in 7/15/30 days
#in_timeframe = pd.to_datetime(pd.date_range(date.today() - timedelta(days = 7), date.today()))
#rolelogin_raw[rolelogin_raw['date'].isin(in_timeframe)]

In [11]:
#select the columan 'userid','date' of specific userid from rolelogin_raw then deduplicate
# AAA = (rolelogin_raw.loc[rolelogin_raw['userid'] == 13726108])[['userid','date']].drop_duplicates()

# start_date = pd.to_datetime(date.today() - timedelta(days = 180))
# end_date = pd.to_datetime(date.today())
    
# AAA.loc[AAA['date'].between(start_date, end_date, inclusive=True)]



In [12]:
#check the data with random userid
# rows = np.random.choice(login_days_final.index.values, 20)
# sampled_df = login_days_final.loc[rows]
#sampled_df
#print (sampled_df)
#userid = 13333645

In [13]:
#consecutive login days with 7,15,30 days
days_set = [7,15,30]

#select the column we need
consecu_login = rolelogin_raw[['userid','roleid','date']].drop_duplicates().reset_index(drop=True)

#order the data by date
sorted_consecu_login = consecu_login.groupby(['userid','roleid']).apply(lambda x : x.sort_values(by = ['date'], ascending = True))

#add column with previous date by group
sorted_consecu_login['last_login'] = sorted_consecu_login.groupby(['userid','roleid'])['date'].shift()

#drop na with .dropna()
sorted_consecu_login = sorted_consecu_login.dropna()

#add a column of the difference between date and last_login, and access the .days attribute by dt.days
sorted_consecu_login['diff_login'] = (sorted_consecu_login['date'] - sorted_consecu_login['last_login']).dt.days

In [14]:
#sorted_consecu_login.loc[(sorted_consecu_login['userid'] == 3922331) & (sorted_consecu_login['diff_login'] == 1)]


consecu_login_result = pd.DataFrame(rolelogin_raw[['userid','roleid']]).drop_duplicates()
for i in days_set:
    start_date = pd.to_datetime(date.today() - timedelta(days = i))
    #start_date = pd.to_datetime(date.today() - timedelta(days = i+45))
    end_date = pd.to_datetime(date.today() - timedelta(days = 1))
    #end_date = pd.to_datetime(date.today() - timedelta(days = 8))
    
    con_temp_1 = sorted_consecu_login.loc[sorted_consecu_login['date'].between(start_date, end_date, inclusive=True)]
    
    con_temp_2 = pd.DataFrame(con_temp_1.loc[con_temp_1['diff_login'] == 1].groupby(['userid','roleid'])['date'].nunique())
    
    con_temp_2.columns = ['consecutive_login_days' + '_in_' + str(i) + '_days']
    
       
    consecu_login_result = pd.merge(consecu_login_result,  con_temp_2, how = 'outer', left_on = ['userid','roleid'], right_on = ['userid','roleid'])
    
consecu_login_result = consecu_login_result.fillna(0)
consecu_login_result = consecu_login_result[consecu_login_result['userid'].isin(login_days_final['userid'])]
#print(consecu_login_result)

consecu_login_result.to_csv("../output/g1jp_user_consecutive_login_days.csv", sep = ',', index = False)

In [15]:
#read the order data
xga_orders = pd.read_csv("/home/xl_daily_update/raw_data/cpf/xga_orders.csv")

g1jp_orders = (xga_orders.loc[xga_orders['app_id'] == 1003])[['gid','price','regdate']]

g1jp_orders['date'] = pd.to_datetime(g1jp_orders['regdate']).dt.date

g1jp_orders = g1jp_orders.rename(columns={'gid':'userid'})

#g1jp_orders.head()

/tmp/ipykernel_23180/3700482672.py:2: DtypeWarning: Columns (4,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  xga_orders = pd.read_csv("/home/xl_daily_update/raw_data/cpf/xga_orders.csv")


In [16]:
#cumulative payment
cumulative_payment = pd.DataFrame(g1jp_orders.groupby('userid')['price'].sum())
cumulative_payment.columns = ['cumulative_payment']
cumulative_payment = cumulative_payment.reset_index()
cumulative_payment.to_csv("../output/g1jp_user_cumulative_payment.csv", sep = ',', index = False)


In [17]:
#cumulative payment within 7,15,30 days
days_set = [7,15,30]

cumulative_payment_inframe = pd.DataFrame(rolelogin_raw['userid']).drop_duplicates()
for i in days_set:
    start_date = pd.to_datetime(date.today() - timedelta(days = i))
    #start_date = pd.to_datetime(date.today() - timedelta(days = i+45))
    end_date = pd.to_datetime(date.today() - timedelta(days = 1))
    #end_date = pd.to_datetime(date.today() - timedelta(days = 8))
    
    cu_pay_1 = g1jp_orders.loc[g1jp_orders['date'].between(start_date, end_date, inclusive = True)]
    
    cu_pay_2 = pd.DataFrame(cu_pay_1.groupby('userid')['price'].sum())
    
    cu_pay_2.columns = ['cumulative_payment_in_' + str(i) + '_days' ]
    
    cumulative_payment_inframe = pd.merge(cumulative_payment_inframe, cu_pay_2, how = "outer", on = ['userid','userid'])
    
cumulative_payment_inframe_result = cumulative_payment_inframe.fillna(0)
#print(cumulative_payment_inframe_result)
cumulative_payment_inframe_result.to_csv("../output/g1jp_user_cumulative_payment_inframe.csv", sep = ',', index = False)
    

In [18]:
# start_date = pd.to_datetime(date.today() - timedelta(days = 7))
# end_date = pd.to_datetime(date.today() - timedelta(days = 1))
# date_range = pd.period_range(start_date, end_date)
# #date_range

In [19]:
#the number of payment in 7,15,30 days
days_set = [7,15,30]


payment_frequency = pd.DataFrame(rolelogin_raw['userid']).drop_duplicates()
for i in days_set:
    start_date = pd.to_datetime(date.today() - timedelta(days = i))
    #start_date = pd.to_datetime(date.today() - timedelta(days = i+45))
    end_date = pd.to_datetime(date.today() - timedelta(days = 1))
    #end_date = pd.to_datetime(date.today() - timedelta(days = 8))

    selected_payment = g1jp_orders[g1jp_orders['date'].between(start_date, end_date, inclusive=True)]

    #overall payment times in selected time frame
    payment_count = pd.DataFrame(selected_payment.groupby('userid').size())
    
    #rename
    payment_count.columns = ['payment_count_in_' + str(i) + '_days']

    #we are going to count the payment frequency by date
    payment_frequency_temp = pd.DataFrame(round(i/payment_count))
    
    payment_frequency_temp.columns = ['payment_frequency_in_' + str(i) + '_days']
    
    
    count_frequency =   pd.merge(payment_frequency, payment_count, how = "outer", on = ['userid','userid'])
    
    payment_frequency = pd.merge(count_frequency, payment_frequency_temp, how = "outer", on = ['userid', 'userid'])
    
payment_frequency = payment_frequency.fillna(0)
#print(payment_frequency)

payment_frequency.to_csv("../output/g1jp_user_payment_frequency.csv", sep = ',', index = False)
    

In [20]:
start_time = "2022-04-19"
end_time = date.today()

logname = 'shoptrade'

shoptrade_id_list = [60134,60136,60137,60138,60147,60146,60163,60139,60154,60149,60172,60161,60203,60201,60176,60177,60191,60235,60237,60252,60253,60265,60266,60270,60251,60272,60286,60296,10029,64063,64059,60121,60123,60122,10026,10030,10026,10027,10028,41006,41008,41055,10033,10034,10048,10019,10020,10442,1,32,34,60118,60119,60120,60121,60140,60141,64101,60456,60148,64063,59998,59999,60000,60111,60112,60113,60114,60115]

gacha_id_list = [62001,62005,62003,62004]

card_fortune_store_id_list = [210,810,1300,1400,4050,4160,4170,4200,4210,4220,4230,4800,4810]

male_outfit_fortune_store_id_list = [51002,51014,51050,51032,51026]

female_outfit_fortune_store_id_list = [51003,51015,51051,51033,51027]

shoptrade_id = pd.DataFrame()
gacha_id = pd.DataFrame()
card_fortune_store_id = pd.DataFrame()
male_outfit_fortune_store_id = pd.DataFrame()
female_outfit_fortune_store_id = pd.DataFrame()


for i in pd.period_range(start_time, end_time):
    data_path = glob.glob('/home/xl_daily_update/raw_data/serverlog/G1JP/' + '/*/' + logname + '.log.' + str(i) + '*', recursive = True)
    
    for j in data_path:
        with xz.open(j, 'rt', encoding = 'utf-8') as f:
            
            df_shoptrade = pd.read_csv(f, header = None, sep = '\x01', usecols = [1,9,10,13], names = ["datetime", "userid", "roleid",'shoptrade_id'])
            df_shoptrade = df_shoptrade[(df_shoptrade['userid'].isin(mainrole.userid)) & (df_shoptrade['roleid'].isin(mainrole.roleid))]
            
            df_shoptrade_item = df_shoptrade[df_shoptrade['shoptrade_id'].isin(shoptrade_id_list)]
            shoptrade_id = pd.concat([shoptrade_id,df_shoptrade_item])
            
            
            df_gacha = df_shoptrade[df_shoptrade['shoptrade_id'].isin(gacha_id_list)]
            gacha_id = pd.concat([gacha_id,df_gacha])
            
            
            df_card_fortune_store = df_shoptrade[df_shoptrade['shoptrade_id'].isin(card_fortune_store_id_list)]
            card_fortune_store_id = pd.concat([card_fortune_store_id,df_card_fortune_store])
            
            
            df_male_outfit = df_shoptrade[df_shoptrade['shoptrade_id'].isin(male_outfit_fortune_store_id_list)]
            male_outfit_fortune_store_id = pd.concat([male_outfit_fortune_store_id,df_male_outfit])
            
            
            df_female_outfit = df_shoptrade[df_shoptrade['shoptrade_id'].isin(female_outfit_fortune_store_id_list)]
            female_outfit_fortune_store_id = pd.concat([female_outfit_fortune_store_id,df_female_outfit])


In [21]:
shoptrade_id_f = shoptrade_id.groupby(['userid','roleid',])['shoptrade_id'].nunique().reset_index().rename(columns = {'shoptrade_id' : "item_buy_times"})
shoptrade_id_f = shoptrade_id_f[shoptrade_id_f['userid'].isin(login_days_final['userid'])]
shoptrade_id_f.to_csv("../output/g1jp_user_shoptrade_item_buy_times.csv", index = False)

gacha_id_f = gacha_id.groupby(['userid','roleid',])['shoptrade_id'].nunique().reset_index().rename(columns = {'shoptrade_id' : "gacha_buy_times"})
gacha_id_f = gacha_id_f[gacha_id_f['userid'].isin(login_days_final['userid'])]
gacha_id_f.to_csv("../output/g1jp_user_gacha_buy_times.csv", index = False)

card_fortune_store_id_f = card_fortune_store_id.groupby(['userid','roleid',])['shoptrade_id'].nunique().reset_index().rename(columns = {'shoptrade_id' : "card_fortune_store_buy_times"})
card_fortune_store_id_f = card_fortune_store_id_f[card_fortune_store_id_f['userid'].isin(login_days_final['userid'])]
card_fortune_store_id_f.to_csv("../output/g1jp_user_card_fortune_store_buy_times.csv", index = False)

male_outfit_fortune_store_id_f = male_outfit_fortune_store_id.groupby(['userid','roleid',])['shoptrade_id'].nunique().reset_index().rename(columns = {'shoptrade_id' : "male_outfit_fortune_store_buy_times"})
male_outfit_fortune_store_id_f = male_outfit_fortune_store_id_f[male_outfit_fortune_store_id_f['userid'].isin(login_days_final['userid'])]
male_outfit_fortune_store_id_f.to_csv("../output/g1jp_user_male_outfit_fortune_store_buy_times.csv", index = False)

female_outfit_fortune_store_id_f = female_outfit_fortune_store_id.groupby(['userid','roleid',])['shoptrade_id'].nunique().reset_index().rename(columns = {'shoptrade_id' : "female_outfit_fortune_buy_times"})
female_outfit_fortune_store_id_f = female_outfit_fortune_store_id_f[female_outfit_fortune_store_id_f['userid'].isin(login_days_final['userid'])]
female_outfit_fortune_store_id_f.to_csv("../output/g1jp_user_female_outfit_fortune_store_buy_times.csv", index = False)

In [22]:
#print(shoptrade_id_f)

In [23]:
#calculate the frequency of buying shoptrade items. Treat the date with buying as 1, other as 0
#In other words, compute the number of dates of processed shoptrade data
shoptrade_id_date = shoptrade_id.copy()
shoptrade_id_date['date'] = pd.to_datetime(shoptrade_id_date['datetime']).dt.date

shoptrade_id_date = pd.DataFrame(shoptrade_id_date.groupby(['userid','roleid'])['date'].nunique()).reset_index().rename(columns = {'date' : 'buy_times'})

days_from_OB = (date.today() - date(2022,7,19)).days

shoptrade_id_date['frequency'] = pd.DataFrame(shoptrade_id_date.groupby(['userid','roleid'])['buy_times']
                                              .apply(lambda buy_times : round(days_from_OB / buy_times)))

shoptrade_id_date = shoptrade_id_date[shoptrade_id_date['userid'].isin(login_days_final['userid'])]
#shoptrade_id_date = shoptrade_id_date.reset_index()
#shoptrade_id_date
shoptrade_id_date.to_csv("../output/g1jp_user_shoptrade_item_buy_frequency.csv", index = False)


In [24]:
# paid_user_target = cumulative_payment[(cumulative_payment['cumulative_payment'] > 0) & (cumulative_payment['cumulative_payment'] < 30000)]
# paid_user_target

In [25]:
#login_days_final[login_days_final['userid'].isin(paid_user_target['userid'])]